# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [97]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [68]:
#Loading CSV
data = "../WeatherPy/Output/WeatherDF.csv"
data = pd.read_csv(data)
#Drop N/A
data = data.dropna(how='any')
# Convert into Dataframe
data = pd.DataFrame(data)
print(data.dtypes)
data.count()


Unnamed: 0      int64
City           object
Country        object
Lat           float64
Lng           float64
Max Temp      float64
Humidity      float64
Cloudiness    float64
Wind Speed    float64
Date          float64
dtype: object


Unnamed: 0    526
City          526
Country       526
Lat           526
Lng           526
Max Temp      526
Humidity      526
Cloudiness    526
Wind Speed    526
Date          526
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [69]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Assign Lat and Lng as location
location = data[["Lat","Lng"]]

# Heat layer with Humidity as weight
humidity = data["Humidity"]
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=True, max_intensity=100,
                                 point_radius = 5)

# Add the layer to the map
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

##### 

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [90]:
Vacation = pd.DataFrame(data, columns = ["City","Country","Lat", "Lng", "Max Temp", "Wind Speed", "Cloudiness"])

# Filters
max_temp = (Vacation["Max Temp"] <= 80) & (Vacation["Max Temp"] > 70)
wind_speed = Vacation["Wind Speed"] < 10
cloudiness = Vacation["Cloudiness"] == 0

# New dataframe
VacationDF = Vacation[max_temp & wind_speed & cloudiness]
VacationDF.head()


,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness
148,carnarvon,AU,-24.8667,113.6333,73.40,5.75,0.0
183,abu kamal,SY,34.4506,40.9171,73.27,9.71,0.0
227,la reforma,MX,25.0833,-108.0500,77.00,5.99,0.0
389,inhambane,MZ,-23.8650,35.3833,72.88,4.90,0.0
419,karatau,KZ,43.1667,70.4667,72.00,2.80,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [102]:
hotel_df = VacationDF
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# For loop through dataframe to search google places
for index, row in hotel_df.iterrows():
    # Get location via data frame
    lat = row["Lat"]
    lng = row["Lng"]

    # Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # URL to pull from
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and print url
    name_address = requests.get(base_url, params=params)
    # Convert to json
    name_address = name_address.json()
    
    # Try/Except block to add hotels
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [106]:
hotels_df = hotel_df.dropna()
hotels_df.head()

,City,Country,Lat,Lng,Max Temp,Wind Speed,Cloudiness,Hotel Name
148,carnarvon,AU,-24.8667,113.6333,73.40,5.75,0.0,Hospitality Carnarvon
389,inhambane,MZ,-23.8650,35.3833,72.88,4.90,0.0,Sentidos Beach Retreat


In [107]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]

In [108]:
# Plot markers on top of the heatmap
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add marker layer ontop of heat map
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…